In [57]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import PreTrainedTokenizerFast
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
import asyncio
import aiohttp
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
nest_asyncio.apply()


In [2]:
def scrape_html(url):
    # Attempt to fetch the page's HTML content using the scraping bee API
    # response = requests.get(url)
    response = requests.get(
        url='https://app.scrapingbee.com/api/v1/',
        params={
            'api_key': 'U9048JRQFEVT5EP5HC2YX6ZHNTK3HGVOP2GGHKIGMJ73OXO6GQQFJ2MEV9HY3MPIYUPLO18WIX9WQIUD',
            'url': url,
            'wait': '3000', 
        },
        
    )
    html_content = response.text
    return html_content
insta_html = scrape_html('https://www.instagram.com')

In [60]:

async def fetch_html(session, url, api_key):
    params = {
        'api_key': api_key,
        'url': url,
        'wait': '3000',
    }
    async with session.get('https://app.scrapingbee.com/api/v1/', params=params) as response:
        return await response.text()

async def scrape_html(urls, api_key):
    connector = aiohttp.TCPConnector(limit_per_host=5)  # Max of 5 concurrent connections to the same endpoint
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for url in urls:
            task = asyncio.create_task(fetch_html(session, url, api_key))
            tasks.append(task)

        html_contents = []
        for f in tqdm_asyncio.as_completed(tasks):
            result = await f
            html_contents.append(result)
        return html_contents

# List of URLs to test
urls = ['https://www.instagram.com' for _ in range(20)]  # Example: replace with your actual URLs
api_key = 'U9048JRQFEVT5EP5HC2YX6ZHNTK3HGVOP2GGHKIGMJ73OXO6GQQFJ2MEV9HY3MPIYUPLO18WIX9WQIUD'

# Run the async scrape function and print the length of each response
async def main():
    htmls = await scrape_html(urls, api_key)
    for html in htmls:
        print(len(html))

# if __name__ == '__main__':
    # asyncio.run(main())
htmls = asyncio.run(scrape_html(urls, api_key))

100%|██████████| 20/20 [00:28<00:00,  1.40s/it]


In [64]:
def html_to_text(html_content):
    # Parse the HTML content with Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    # Extract and print all text content from the HTML
    text_content = soup.get_text(separator=' ', strip=True)
    return text_content
[html_to_text(html) for html in htmls][:2]

["Instagram Phone number, username, or email Password Log in or Log in with Facebook Forgot password? Don't have an account? Sign up Get the app. Meta About Blog Jobs Help API Privacy Terms Locations Instagram Lite Threads Contact Uploading & Non-Users Meta Verified English Down chevron icon Afrikaans العربية Čeština Dansk Deutsch Ελληνικά English English (UK) Español (España) Español فارسی Suomi Français עברית Bahasa Indonesia Italiano 日本語 한국어 Bahasa Melayu Norsk Nederlands Polski Português (Brasil) Português (Portugal) Русский Svenska ภาษาไทย Filipino Türkçe 中文(简体) 中文(台灣) বাংলা ગુજરાતી हिन्दी Hrvatski Magyar ಕನ್ನಡ മലയാളം मराठी नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港) Български Français (Canada) Română Српски Українська © 2024 Instagram from Meta",
 "Instagram Phone number, username, or email Password Log in or Log in with Facebook Forgot password? Don't have an account? Sign up Get the app. Meta About Blog Jobs Help API Privacy Terms Locations Instagram Lite

In [65]:
def text_to_chunks(text_content, chunk_size):
    # Split the text content into chunks of a specific size
    chunks = [text_content[i:i+chunk_size] for i in range(0, len(text_content), chunk_size)]
    return chunks
insta_chunks = text_to_chunks(html_to_text(htmls[0]), 100)
insta_chunks

['Instagram Phone number, username, or email Password Log in or Log in with Facebook Forgot password? ',
 "Don't have an account? Sign up Get the app. Meta About Blog Jobs Help API Privacy Terms Locations In",
 'stagram Lite Threads Contact Uploading & Non-Users Meta Verified English Down chevron icon Afrikaans',
 ' العربية Čeština Dansk Deutsch Ελληνικά English English (UK) Español (España) Español فارسی Suomi Fr',
 'ançais עברית Bahasa Indonesia Italiano 日本語 한국어 Bahasa Melayu Norsk Nederlands Polski Português (Bras',
 'il) Português (Portugal) Русский Svenska ภาษาไทย Filipino Türkçe 中文(简体) 中文(台灣) বাংলা ગુજરાતી हिन्दी ',
 'Hrvatski Magyar ಕನ್ನಡ മലയാളം मराठी नेपाली ਪੰਜਾਬੀ සිංහල Slovenčina தமிழ் తెలుగు اردو Tiếng Việt 中文(香港',
 ') Български Français (Canada) Română Српски Українська © 2024 Instagram from Meta']

In [5]:
# use the opensearch amazon model to create sparse embeddings of documents etc.
model_path = 'opensearch-neural-sparse-encoding-v1.pt'
model = torch.jit.load(model_path)
model.to('cuda')
tokenizer_file = 'tokenizer.json'
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)

In [11]:
def chunks_to_sparse(descs):
    t = tokenizer(descs, return_tensors='pt', padding='max_length', truncation=True, max_length=64)
    t.to('cuda')
    with torch.no_grad():
        vecs = model(t)['output']
    np_vecs = vecs.cpu().detach().numpy()
    rfs = []
    for vec in np_vecs:
        indices = np.argwhere(vec > 0).flatten()
        tokens = tokenizer.convert_ids_to_tokens(indices)
        values = vec[indices]
        rfs.append(dict(zip(tokens, values)))
    return np_vecs, rfs
svecs = chunks_to_sparse(['oscar is a loyal farmer', 'jeremy is a hard working accountant'])
svecs[0][0].shape, svecs[1][0]

((30522,),
 {'a': 0.080649205,
  'the': 0.003515019,
  'he': 0.12468159,
  'his': 0.58050567,
  'him': 0.18335392,
  'who': 0.42253754,
  'film': 0.74905515,
  'good': 0.2153714,
  'army': 0.0756729,
  'award': 1.2902812,
  'story': 0.22064964,
  'royal': 0.13172622,
  'friend': 0.21865188,
  'character': 0.5295071,
  'awards': 0.099501364,
  'producer': 0.048129253,
  'movie': 0.3746207,
  'actor': 0.6948534,
  'champion': 0.32281154,
  'irish': 0.25947887,
  'drama': 0.09200079,
  'farm': 1.8395033,
  'dog': 0.46629632,
  'comedy': 0.07766671,
  'animal': 0.22657175,
  'rich': 0.078738034,
  'brand': 0.0077574183,
  'classic': 0.04152609,
  'bird': 0.0519735,
  'khan': 0.15939452,
  'victor': 0.3428981,
  'kelly': 0.21804157,
  'agriculture': 0.14154153,
  'hero': 0.18328653,
  'boss': 0.07625518,
  'ross': 0.07878541,
  'stewart': 0.21082355,
  'jerry': 0.1764886,
  'personality': 0.19048817,
  'farmers': 1.3556609,
  'butler': 0.045699302,
  'revenge': 0.31632808,
  'alien': 0.1500

In [7]:
# we need to persist to pg and query pg
# Connect to your postgres DB
conn = psycopg2.connect(
    host="localhost",
    port="5442",
    database="dbname",
    user="user",
    password="password"
)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a query
cur.execute("SELECT version();")
# Retrieve query result
version = cur.fetchone()
print("Connected to:", version)
# register vector type
register_vector(conn)

Connected to: ('PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [8]:
# create table id,embedding
cur = conn.cursor()
cur.execute('CREATE TABLE items (id serial PRIMARY KEY, embedding vector(3));')

embedding = np.array([1, 2, 3])
cur.execute('INSERT INTO items (embedding) VALUES (%s)', (embedding,))


In [39]:
conn.commit()

In [40]:
cur = conn.cursor()
# create table tabs if not exists
cur.execute('CREATE TABLE IF NOT EXISTS tabs (id serial PRIMARY KEY, url text, embedding svector(30522));')
# insert svecs[0] into tabs
cur.executemany('INSERT INTO tabs (url, embedding) VALUES (%s, %s)', [('o', svecs[0][0]), ('j', svecs[0][1])])

In [53]:
qsvec = chunks_to_sparse('watching the computer screen')[0][0]
# N.B. <-> is euclidean which we don't want. fields returns j for euclidean and o for inner product.
# N.B. the documentation says to multiply by -1 but this is not necessary.
# cur.execute('SELECT url, embedding FROM tabs ORDER BY embedding <-> %s LIMIT 1', (qsvec,))
cur.execute('SELECT url, embedding FROM tabs ORDER BY (embedding <#> %s) LIMIT 1', (qsvec,))
cur.fetchone()

('j',
 '[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [67]:
## CREATE TABLES
# Table schema
# tabs: url | title | id | image bytea | status (open/bkmk/hist/web/del) | last_visited | screenshot bytea
# chunks: id | tab_id | text | embedding svector

conn.commit()
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS tabs (id serial PRIMARY KEY, url text, title text, image bytea, status text, last_visited timestamp, screenshot bytea);')
cur.execute('CREATE TABLE IF NOT EXISTS chunks (id serial PRIMARY KEY, url text, text text, embedding svector);')
conn.commit()


In [70]:
def ingest_urls(urls):
    htmls = asyncio.run(scrape_html(urls, api_key))
    texts = [html_to_text(html) for html in htmls]
    tabs = [{'url': url, 'text': text, 'chunks': text_to_chunks(text, 100)} for url, text in zip(urls, texts)]
    chunks = [{'url': tab['url'], 'text': chunk} for tab in tabs for chunk in tab['chunks']]
    npvecs, svecs = chunks_to_sparse([chunk['text'] for chunk in chunks])
    chunks = [{**chunk, 'embedding': npvec} for chunk, npvec in zip(chunks, npvecs)]
    # start new transaction
    conn.commit()
    cur = conn.cursor()
    # insert into tabs
    cur.executemany('INSERT INTO tabs (url) VALUES (%s)', [(tab['url'],) for tab in tabs])
    # insert into chunks
    cur.executemany('INSERT INTO chunks (url, text, embedding) VALUES (%s, %s, %s)', [(chunk['url'], chunk['text'], chunk['embedding']) for chunk in chunks])
    cur.close()
    conn.commit()

urls = ['https://www.instagram.com', 'https://www.facebook.com']
ingest_urls(urls)



100%|██████████| 2/2 [00:05<00:00,  3.00s/it]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
